In [188]:
import numpy as np
import math
PI = math.pi

In [189]:
def init_GMM(K):
    """
        k - k models
    """
    parameters = {}
    parameters['alpha'] = np.random.dirichlet((1, 1), 1)
    parameters['mean'] = np.random.rand(K), 
    parameters['variance'] = np.random.rand(K, )

    return parameters

In [190]:
def tile(y, K):
    return np.tile(y, (1, K))

In [191]:
def Gaussian_func(y, mean, var):
    """
        Params: y - array of outcomes                [N, ]
                mean - array of mean of K models       [K, ]
                var - array of variance of K models    [K, ]
    """
    N = len(y)
    K = len(mean)
    y = y.reshape(N, 1)
    y_tiled = tile(y, K)


    exponent = np.exp(-(y_tiled - mean)**2 / (2 * var))
    return exponent / (np.sqrt(2 * PI * var))

In [192]:
print("========================for test=========================")
rng = np.random.default_rng(seed = 2021615)
y_test = rng.random((5, 1))
mean_test = rng.random((2, ))
var_test = rng.random((2, ))
print("y:", y_test)
print("mean:", mean_test)
print("var:", var_test)
res_test = Gaussian_func(y_test, mean_test, var_test)
print("res:", res_test)

========================for test=========================
y: [[0.0800576 ]
 [0.2960229 ]
 [0.90397203]
 [0.51955655]
 [0.88693269]]
mean: [0.9633458  0.93267767]
var: [0.05532568 0.19115782]
res: [[1.46978860e-03 1.36275234e-01]
 [3.03120323e-02 3.16065430e-01]
 [1.64289752e+00 9.10495873e-01]
 [2.86051255e-01 5.83903610e-01]
 [1.60890121e+00 9.07479696e-01]]


In [193]:
def Expectation(parameters, y):
    """
        Params: parametrs - dict containing parameters for k distribute Gaussian models
                y - outcomes
        Yield: res - matrix containing response for jth outcome with kth model
    """
    alpha, mean, variance = parameters['alpha'], parameters['mean'], parameters['variance']
    N = len(y)
    K = len(alpha)
    # res = np.zeros((N, K))
    # total_prob = np.zeros(N)

    res = alpha * Gaussian_func(y, mean, variance)
    total_prob = np.sum(res, axis = 1).reshape(N, 1)
    res = res / total_prob

    return res

In [194]:
print("========================for test=========================")
parameters_test = {
    "alpha" : np.array([0.5, 0.5]),
    "mean" : mean_test,
    "variance" : var_test
}
print("GF", res_test)
print("total_prob:", np.sum(res_test, axis = 1).reshape(5, 1))
print("res:", Expectation(parameters_test, y_test))

========================for test=========================
GF [[1.46978860e-03 1.36275234e-01]
 [3.03120323e-02 3.16065430e-01]
 [1.64289752e+00 9.10495873e-01]
 [2.86051255e-01 5.83903610e-01]
 [1.60890121e+00 9.07479696e-01]]
total_prob: [[0.13774502]
 [0.34637746]
 [2.55339339]
 [0.86995486]
 [2.51638091]]
res: [[0.01067036 0.98932964]
 [0.08751156 0.91248844]
 [0.64341731 0.35658269]
 [0.3288116  0.6711884 ]
 [0.63937109 0.36062891]]


In [195]:
def Maximization(res, y, parameters):
    """
        Params: res - res for jth outcome with kth model (matrix in shape of [N, K])
                y - outcomes (array)
        Yield: a new set of parameters for k models
    """
    N = len(y)
    K = res.shape[1]

    mean = parameters['mean']
    res_sum = np.sum(res, axis = 0)
    
    mean = np.dot(y.T, res) / res_sum
    variance = np.dot(res.T, np.square(tile(y, K) - mean)).diagonal() / res_sum
    alpha = res_sum / N
    
    parameters = {}
    parameters['mean'] = mean
    parameters['variance'] = variance
    parameters['alpha'] = alpha

    return parameters

In [196]:
print("========================for test=========================")
print("res:", res_test)
print("new parameters: ")
print(Maximization(res_test, y_test, parameters_test))

========================for test=========================
res: [[1.46978860e-03 1.36275234e-01]
 [3.03120323e-02 3.16065430e-01]
 [1.64289752e+00 9.10495873e-01]
 [2.86051255e-01 5.83903610e-01]
 [1.60890121e+00 9.07479696e-01]]
new parameters: 
{'mean': array([[0.85998534, 0.71325257]]), 'variance': array([0.01345599, 0.06728903]), 'alpha': array([0.71392636, 0.57084397])}


In [197]:
def GMM_EM(y, K):
    parameters = init_GMM(K)
    hist = []

    for epoch in range(3):
        res = Expectation(parameters, y)
        parameters = Maximization(res, y, parameters)
        hist.append(parameters)

    return hist

In [198]:
o = np.array([-67, -48, 6, 8, 14, 16, 23, 24, 28, 29, 41, 49, 56, 60, 75]).reshape(15, 1)
o = (o - np.sum(o)) / np.var(o)
print(o)
hist = GMM_EM(o, 2)
hist

[[-0.28653894]
 [-0.27224959]
 [-0.23163778]
 [-0.23013363]
 [-0.22562121]
 [-0.22411707]
 [-0.21885257]
 [-0.2181005 ]
 [-0.21509222]
 [-0.21434015]
 [-0.2053153 ]
 [-0.19929874]
 [-0.19403424]
 [-0.19102596]
 [-0.1797449 ]]


[{'mean': array([[-0.22039643, -0.2205897 ]]),
  'variance': array([0.00075178, 0.0007571 ]),
  'alpha': array([0.94608695, 0.05391305])},
 {'mean': array([[-0.22039119, -0.22068166]]),
  'variance': array([0.00075141, 0.00076359]),
  'alpha': array([0.94608565, 0.05391435])},
 {'mean': array([[-0.22037915, -0.22089291]]),
  'variance': array([0.00075078, 0.0007745 ]),
  'alpha': array([0.94608094, 0.05391906])}]

In [199]:
from sklearn.mixture import GaussianMixture

In [200]:
gm = GaussianMixture(2).fit(o)
print(gm.means_)
print(gm.covariances_)
print(gm.weights_)

[[-0.21134358]
 [-0.27940284]]
[[[2.43925784e-04]]

 [[5.20462599e-05]]]
[0.86683257 0.13316743]
